In [ ]:
import csaf.config as cconf
import csaf.system as csys
import numpy as np

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_shield_config.toml")

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 35.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt

from sympy.utilities.lambdify import lambdify, implemented_function

In [ ]:
x, y, theta_f, tau, x1, x2, xp1, xp2 = sympy.symbols('x y theta_f tau x1 x2 xp1 xp2', real=True)

def delta(x, v=0.0):
  return 0 if x != v else 100
modules = [{'Heaviside': lambda x: np.heaviside(x, 1), 'DiracDelta': delta}, 'numpy']


def kernel_preprocess(f):
  def inner(t, tt):
    t = [t] if isinstance(t, float) else t
    tt = [tt] if isinstance(tt, float) else tt
    return build_kernel_matrix(f, t, tt)
  return inner


def build_kernel_matrix(k, t, tt):
  """build out a kernel matrix from a matrix or scalar valued kernel function k"""
  K = np.empty((len(t), len(tt))).tolist()
  for idx, ti in enumerate(t):
    for jdx, tti in enumerate(tt):
      val = k(ti, tti)
      K[idx][jdx] = val
  reta = np.array(K)
  # case: matrix valued kernel
  if len(reta.shape) == 4:
    reta = np.block(K)
  # case normal
  elif len(reta.shape) == 2:
    pass
  else:
    raise RuntimeError(f"illegal kernel return shape {ret.shape}")
  return reta



def derivative_constrain(kern_expr):
  """constrain a GP with a derivative df/dx - f' = 0"""
  def make_func(expr):
    f = lambdify((x, y, theta_f, tau), expr, modules=modules)
    return f

  f00 = make_func(kern_expr)
  f01 = make_func(sympy.diff(kern_expr, x))
  f10 = make_func(sympy.diff(kern_expr, y))
  f11 = make_func(sympy.diff(sympy.diff(kern_expr, x), y))
  def inner(theta_f, tau):
    @kernel_preprocess
    def _k(x, y):
      return np.array([[f00(x, y, theta_f, tau), f01(x, y, theta_f, tau), 0, 0],
                       [f10(x, y, theta_f, tau), f11(x, y, theta_f, tau), 0, 0],
                      [0, 0, f00(x, y, theta_f, tau), f01(x, y, theta_f, tau)],
                      [0, 0, f10(x, y, theta_f, tau), f11(x, y, theta_f, tau)]])
    return _k
  return inner

def circular_constrain(kern_expr):
  """constrain a GP with a derivative df/dx - f' = 0"""
  def make_func(expr):
    f = lambdify((x, y, theta_f, tau), expr, modules=modules)
    return f

  kf = make_func(kern_expr)
  kdf = make_func(sympy.diff(kern_expr, x))
  dkf = make_func(sympy.diff(kern_expr, y))
  dkdf = make_func(sympy.diff(sympy.diff(kern_expr, x), y))
  def inner(tau, theta_f, theta_d):
    @kernel_preprocess
    def _k(t, tt):
      return np.block([[dkdf(t, tt, theta_f, tau)+3*kf(t, tt, theta_f, tau), kdf(t, tt, theta_f, tau), 0, kf(t, tt, theta_f, tau)], 
                    [dkf(t, tt, theta_f, tau), dkdf(t, tt, theta_f, tau)+kf(t, tt, theta_f, tau), -kf(t, tt, theta_f, tau), 0],
                     [0, -kf(t, tt, theta_f, tau),  dkdf(t, tt, theta_f, tau)+3*kf(t, tt, theta_f, tau), kdf(t, tt, theta_f, tau)],
                     [kf(t, tt, theta_f, tau), 0, dkf(t, tt, theta_f, tau), dkdf(t, tt, theta_f, tau)+kf(t, tt, theta_f, tau)]])
      #return np.block([[dkdf(t, tt, theta_f, tau)+2*kf(t, tt, theta_f, tau), 0, 0, kf(t, tt, theta_f, tau)], 
      #              [0, kf(t, tt, theta_f, tau), -kf(t, tt, theta_f, tau), 0],
      #               [0, -kf(t, tt,theta_f, tau), dkdf(t, tt,theta_f, tau)+2*kf(t, tt, theta_f, tau),0],
      #              [kf(t, tt,theta_f, tau), 0, 0, kf(t, tt,theta_f, tau)]])
    return _k
  return inner


def dubins_constrain(kern_expr, u):
  """constrain a GP with a derivative df/dx - f' = 0"""
  def make_func(expr):
    f = lambdify((x, y, theta_f, tau), expr, modules=modules)
    return f

  kf = make_func(kern_expr)
  kdf = make_func(sympy.diff(kern_expr, x))
  dkf = make_func(sympy.diff(kern_expr, y))
  dkdf = make_func(sympy.diff(sympy.diff(kern_expr, x), y))
  dk2df2 = make_func(sympy.diff(sympy.diff(kern_expr, x, 2), y, 2))
  def inner(tau, theta_f, theta_d):
    @kernel_preprocess
    def _k(t, tt):
      return np.block([[u**4*kf(t, tt, theta_f, tau), 0, 0, -u**2*dkdf(t, tt, theta_f, tau)],
                       [0, dk2df2(t, tt, theta_f, tau) + dkdf(t, tt, theta_f, tau)+u**2*kf(t, tt, theta_f, tau), u**2*dkdf(t, tt, theta_f, tau), 0],
                       [0, u**2*dkdf(t, tt, theta_f, tau), u**4*kf(t, tt, theta_f, tau), 0],
                       [-u**2*dkdf(t, tt, theta_f, tau), 0, 0, dk2df2(t, tt, theta_f, tau) + dkdf(t, tt, theta_f, tau)+u**2*kf(t, tt, theta_f, tau)]])
      #return np.block([[dkdf(t, tt, theta_f, tau)+2*kf(t, tt, theta_f, tau), 0, 0, kf(t, tt, theta_f, tau)], 
      #              [0, kf(t, tt, theta_f, tau), -kf(t, tt, theta_f, tau), 0],
      #               [0, -kf(t, tt,theta_f, tau), dkdf(t, tt,theta_f, tau)+2*kf(t, tt, theta_f, tau),0],
      #              [kf(t, tt,theta_f, tau), 0, 0, kf(t, tt,theta_f, tau)]])
    return _k
  return inner



def make_kf(tau, thetaf):
  """cubic spline covariance kernel"""
  @kernel_preprocess
  def k(t, tt):
    return thetaf**2 * (
        (1.0/3.0) * min(t + tau, tt + tau)**3 + \
        (1.0/2.0) * np.abs(t - tt) * min(t + tau, tt + tau)**2
    )
  return k


def make_kdf(thetaf, thetafp):
  @kernel_preprocess
  def kdf(t, tt):
    return thetaf * thetafp * (
        (1.0 if t < tt else 0.0) * (t**2) / 2.0 + \
        (1.0 if t >= tt else 0.0) * (t * tt - (tt**2.0) / 2.0)
    )
  return kdf


def make_dkf(thetaf, thetafp):
  #f = make_kdf(thetaf, thetafp)
  @kernel_preprocess
  def dkf(t, tt):
    return thetaf * thetafp * (
        (1.0 if tt < t else 0.0) * (tt**2) / 2.0 + \
        (1.0 if tt >= t else 0.0) * (t * tt - (t**2.0) / 2.0)
    )
    return f(tt, t)
  return dkf


def make_dkdf(thetafp):
  @kernel_preprocess
  def dkdf(t, tt):
    return thetafp**2 * min(t, tt)
  return dkdf


def make_indk(tau, thetaf, thetafp):
  kf = make_kf(tau, thetaf)
  kdf = make_kdf(thetaf, thetafp)
  dkf = make_dkf(thetaf, thetafp)
  dkdf = make_dkdf(thetafp)
  @kernel_preprocess
  def k(t, tt):
    return np.block([[kf(t, tt), kdf(t, tt), 0, 0], 
                    [dkf(t, tt), dkdf(t, tt), 0, 0],
                     [0, 0, kf(t, tt), kdf(t, tt)],
                     [0, 0, dkf(t, tt), dkdf(t, tt)]])
  return k


def make_k(tau, thetaf, thetafp):
  kf = make_kf(tau, thetaf)
  kdf = make_kdf(thetaf, thetafp)
  dkf = make_dkf(thetaf, thetafp)
  dkdf = make_dkdf(thetafp)
  @kernel_preprocess
  def k(t, tt):
    return np.block([[dkdf(t, tt)+2*kf(t, tt), 0, 0, kf(t, tt)], 
                    [0, kf(t, tt), -kf(t, tt), 0],
                     [0, -kf(t, tt), dkdf(t, tt)+2*kf(t, tt),0],
                    [kf(t, tt), 0, 0, kf(t, tt)]])
  return k


In [ ]:
expr = theta_f**2*(1/3*sympy.Min(x+tau, y+tau)**3 + 1/2*sympy.Abs(x-y)*sympy.Min(x+tau, y+tau)**2)
#expr = theta_f**2*(sympy.exp(-(x-y)**2/tau))
expr

In [ ]:
k = dubins_constrain(expr, 10.0)(11, 1, 1)
#k = derivative_constrain(expr)(1.0, 100.0)

In [ ]:
trajs["plant"].times[1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
xf, yf = np.array(trajs["plant"].states)[:, 9], np.array(trajs["plant"].states)[:, 11]
#xf = MinMaxScaler().fit_transform(xf[:, np.newaxis]).flatten()
#yf = MinMaxScaler().fit_transform(yf[:, np.newaxis]).flatten()


N = 670
S =0
Td = trajs["plant"].times[S::N]
x = xf[S::N]
y = yf[S::N]
xt = -np.diff(np.hstack((xf, xf[-1])))[S::N]/0.01
yt = -np.diff(np.hstack((yf, yf[-1])))[S::N]/0.01

y[2:] += 1000

#y += (np.random.rand(len(y))*1)

#for i in range(10):
#    Td = np.hstack((Td[0], Td))
#    x = np.hstack((x[0], x))
#    y = np.hstack((y[0], y))
#    xt = np.hstack((xt[0], xt))
#    yt = np.hstack((yt[0], yt))

In [ ]:
plt.plot(xf, yf)
plt.scatter(x, y)
for (yi, zi, yti, zti) in zip(x, y, xt, yt):
    plt.plot([yi-yti, yi+yti], [zi-zti, zi+zti], '-k', LineWidth=2) 

In [ ]:

K = k(Td, Td)
P = np.linalg.inv(K + 0.0001*np.eye(K.shape[0]))
plt.imshow(K[0::4, :][:, 0::4])
plt.imshow(P[0::4, :][:, 0::4])

In [ ]:
def cov(t, k, P):
  return k(t,t) - k(Td, t).T @ P @ k(Td,t)

In [ ]:
tn = np.linspace(0, 40, 100)
cs = [cov(ti, k, P) for ti in tn]

In [ ]:
def conv_striped(P):
  P11 = P[::4, :][:, 0::4]
  P12 = P[::4, :][:, 1::4]
  P13 = P[::4, :][:, 2::4]
  P14 = P[::4, :][:, 3::4]

  P21 = P[1::4, :][:, 0::4]
  P22 = P[1::4, :][:, 1::4]
  P23 = P[1::4, :][:, 2::4]
  P24 = P[1::4, :][:, 3::4]

  P31 = P[2::4, :][:, 0::4]
  P32 = P[2::4, :][:, 1::4]
  P33 = P[2::4, :][:, 2::4]
  P34 = P[2::4, :][:, 3::4]

  P41 = P[3::4, :][:, 0::4]
  P42 = P[3::4, :][:, 1::4]
  P43 = P[3::4, :][:, 2::4]
  P44 = P[3::4, :][:, 3::4]

  return np.block([[P11, P12, P13, P14],
                   [P21, P22, P23, P24],
                   [P31, P32, P33, P34],
                   [P41, P42, P43, P44]])



def mu(t, k, P):
  P11 = P[::4, :][:, ::4]
  P12 = P[::4, :][:, 1::4]
  P13 = P[::4, :][:, 2::4]
  P14 = P[::4, :][:, 3::4]

  P21 = P[1::4, :][:, ::4]
  P22 = P[1::4, :][:, 1::4]
  P23 = P[1::4, :][:, 2::4]
  P24 = P[1::4, :][:, 3::4]

  P31 = P[2::4, :][:, ::4]
  P32 = P[2::4, :][:, 1::4]
  P33 = P[2::4, :][:, 2::4]
  P34 = P[2::4, :][:, 3::4]

  P41 = P[3::4, :][:, ::4]
  P42 = P[3::4, :][:, 1::4]
  P43 = P[3::4, :][:, 2::4]
  P44 = P[3::4, :][:, 3::4]
  rhs = np.vstack((P11 @ np.array(x)[:, np.newaxis] + P12 @ np.array(xt)[:, np.newaxis] + P13 @ np.array(y)[:, np.newaxis] + P14 @ np.array(yt)[:, np.newaxis],
  P21 @ np.array(x)[:, np.newaxis] + P22 @ np.array(xt)[:, np.newaxis] + P23 @ np.array(y)[:, np.newaxis] + P24 @ np.array(yt)[:, np.newaxis],
  P31 @ np.array(x)[:, np.newaxis] + P32 @ np.array(xt)[:, np.newaxis] + P33 @ np.array(y)[:, np.newaxis] + P34 @ np.array(yt)[:, np.newaxis],
  P41 @ np.array(x)[:, np.newaxis] + P42 @ np.array(xt)[:, np.newaxis] + P43 @ np.array(y)[:, np.newaxis] + P44 @ np.array(yt)[:, np.newaxis]))
  kt = k(Td, t)
  lhs = conv_striped(kt)
  return (lhs.T @ rhs).flatten()

In [ ]:
ms = np.array([mu(ti, k, P) for ti in tn])

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(ms[:, 0], ms[:, 2], label="Constrained")
plt.plot(xf, yf, label="Truth")
#plt.plot(msi[:, 0], msi[:, 2], label="Independent")

plt.scatter(x, y, c='k', s=80, label='Training Data')
dt = 0.1
for xi, yi, xti, yti in zip(x, y, xt, yt):
  plt.plot([xi-xti*dt, xi+xti*dt], [yi-yti*dt, yi+yti*dt], '-k', LineWidth=4)

plt.legend()
plt.title("Constrained GPR")
plt.grid()
#plt.axis("equal")